In [25]:
from dotenv import load_dotenv
load_dotenv()

True

In [53]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import AgentEvent, ChatMessage

In [27]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [45]:
# Create the primary agent.
Planner = AssistantAgent(
    "Planner",
    model_client=model_client,
    system_message="""You are Planner Agent Below are your tasks
        1. Give tasks to Coder Agent
        2. Once Task is completed Give code to Tester Agent
        3. Review processes are completed by Coder Agent and Tester Agent
        4. Return Java 17 code with commit message
        5. Once all tasks are completed, Reply with APPROVE to end the task
        6. If Agent replies as DONE move forward with output to next task
        """,
)

# Create the critic agent.
Coder = AssistantAgent(
    "Coder",
    model_client=model_client,
    system_message="Write JAVA17 code for input code provided by Planner agent, Reply with DONE once task is completed",
)

# Create the critic agent.
Tester = AssistantAgent(
    "Tester",
    model_client=model_client,
    system_message="Understand the code provided by coder agent and Write Unit test cases for the code provided, Reply with DONE once task is completed",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([Planner, Coder,Tester], termination_condition=text_termination)

In [46]:
task="""Migrate below code to JAVA17
import java.util.*;
import java.text.SimpleDateFormat;

public class LegacyJavaApp {
    public static void main(String[] args) {
        Date now = new Date();
        SimpleDateFormat sdf = new SimpleDateFormat("yyyy-MM-dd HH:mm:ss");
        System.out.println("Current Date: " + sdf.format(now));

        StringBuffer sb = new StringBuffer();
        sb.append("Hello");
        sb.append(" Java 8");
        System.out.println(sb.toString());

        List<String> names = Arrays.asList("Alice", "Bob", "Charlie");
        Collections.sort(names, new Comparator<String>() {
            @Override
            public int compare(String a, String b) {
                return a.compareTo(b);
            }
        });
        System.out.println("Sorted Names: " + names);
    }
}


"""

In [ ]:
# selector_prompt = """Select from below roles
# {roles}
# for first task select Planner
# Current conversation context:
# {history}

# Read the above conversation, then select an agent from {participants} to perform the next task.
# Make sure the planner agent has assigned tasks before other agents start working.
# Only select one agent.
# """

In [ ]:
from typing import Sequence

def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if not messages:
        return "Planner"  # Start with Planner Agent
    
    last_message = messages[-1]

    if last_message.source == "Planner":
        return "Coder"  # Pass tasks to the coder

    if last_message.source == "Coder":
        return "Tester"  # Coder sends code to the tester

    if last_message.source == "Tester":
        if "error" in last_message.content.lower():
            return "Coder"  # If error occurs, send back to coder
        return "Planner"  # If no error, end process

    return "Planner"  # Default case


In [55]:
team = SelectorGroupChat(
    [Planner, Coder, Tester],
    model_client=model_client,
    termination_condition=text_termination,
    selector_func=selector_func,
    allow_repeated_speaker=False,  # Allow an agent to speak multiple turns in a row.
)

In [57]:
await Console(team.run_stream(task=task))

TypeError: expected string or bytes-like object, got 'AssistantAgent'